### Imports and Preprocessing

In [1]:
# imports
import pandas as pd
import sklearn as skl
import psycopg2
import config as c

# import for multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# connection to database
connection = psycopg2.connect(
    host = c.host,
    port = c.port,
    user = c.user,
    password = c.password,
    database = c.database
    )
cursor=connection.cursor()

In [3]:
# SQL query
data_sql = """
SELECT *
FROM final_dataset;
"""

# load in tables as dataframes
data_df = pd.read_sql(data_sql, con=connection)

In [4]:
# get_dummies to encode HallofFameClass column
data_df = pd.get_dummies(data_df, prefix='HOF', columns=['HallOfFameStatus'])
data_df.drop(['HOF_Not Inducted'], axis=1, inplace=True) # remove extra get_dummies column, drop_first no ideal

# drop coulmns/features
data_df.drop(['GUID',
              'Name',
              'HallofFameClass',
              'YearDrafted',
              'TO_YEAR',
              'Years_Played',
              'HOF_Elgibility_Year',
              'Team',
              'College',
              'GP',
              'MIN',
              'idPlayer',
              'BIRTHDATE',
              'POSITION',
              'AGE_ROOKIE_SEASON',
              'Draft_Decade',
              'numBallotsBeforeInduct'], 
             axis=1, inplace=True)

## Machine Learning

In [5]:
# ML imports
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

In [6]:
# create our features
X = data_df.drop(['HOF_Hall of Fame Member'], axis=1)

# create our target
y = data_df['HOF_Hall of Fame Member']

In [8]:
# normal train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)
Counter(y_train)

# implement ROS resampling
X_resampled, y_resampled = RandomOverSampler(random_state=2, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

# logistic regression using ROS data
model = LogisticRegression(solver='liblinear', random_state=2)
model.fit(X_resampled, y_resampled)

# generate predictions on test set
y_pred_test = model.predict(X_test)

# calculate the accuracy scores
test_acc_score = balanced_accuracy_score(y_test, y_pred_test)
train_acc_score = model.score(X_train, y_train)

# define the confusion matrix
cm = confusion_matrix(y_test, y_pred_test)

# create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Not HOF Member", "HOF Member"], 
    columns=["Predicted Not HOF Member", "Predicted HOF Member"])

# print results
print('_______BASELINE RESULTS_______\n')
print("CONFUSION MATRIX")
display(cm_df)
print("ACCURACY SCORES\n")
print(f'Training Accuracy: {train_acc_score}\n')
print(f'Testing Accuracy: {test_acc_score}\n')
print("CLASSIFICATION REPORT\n")
print(classification_report_imbalanced(y_test, y_pred_test))

Counter({0: 880, 1: 32})

Counter({0: 880, 1: 880})

LogisticRegression(random_state=2, solver='liblinear')

_______BASELINE RESULTS_______

CONFUSION MATRIX


,Predicted Not HOF Member,Predicted HOF Member
Not HOF Member,266,29
HOF Member,2,8


ACCURACY SCORES

Training Accuracy: 0.8826754385964912

Testing Accuracy: 0.8508474576271187

CLASSIFICATION REPORT

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.90      0.80      0.94      0.85      0.73       295
          1       0.22      0.80      0.90      0.34      0.85      0.71        10

avg / total       0.97      0.90      0.80      0.93      0.85      0.73       305

